# *** Compare SFR stages Vs RIV stages ***

- RIV stage comes from the RIV IDF files that were given to us by Deltares.
- Sys 1: DETAILWATERGANGEN (contains the detailed network) will be used for the calculations. The other systems are mostly/completely absent in our region of interest. For proof of that, check ./SFRstage_Vs_RIVstage_NBr43.ipynb 

# 0. Initial

## 0.0. Imports

In [ ]:
import numpy as np

In [1]:
import imod
import flopy

In [2]:
import WS_Mdl.utils as U
import WS_Mdl.utils_imod as UIM
import WS_Mdl.geo as G

In [3]:
import importlib as IL
IL.reload(U)
IL.reload(UIM)
IL.reload(G)

<module 'WS_Mdl.geo' from 'G:\\code\\WS_Mdl\\geo.py'>

In [4]:
import os
from os import listdir as LD, makedirs as MDs
from os.path import join as PJ, basename as PBN, dirname as PDN, exists as PE
import shutil as sh
import pandas as pd
from datetime import datetime as DT
import matplotlib.pyplot as plt
from pathlib import Path
import re
import xarray as xr

In [5]:
import plotly.graph_objects as go
import re

In [6]:
from pathlib import Path
import xarray as xr
import pandas as pd
from tqdm.notebook import tqdm

## 0.1. Options

In [7]:
U.set_verbose(False)

In [8]:
MdlN = 'NBr40'
MdlN_B = 'NBr43'

In [9]:
d_Pa = U.get_MdlN_Pa(MdlN)
d_INI = U.INI_to_d(d_Pa['INI'])
Xmin, Ymin, Xmax, Ymax, cellsize, N_R, N_C = U.Mdl_Dmns_from_INI(d_Pa['INI'])
SP_date_1st = DT.strftime(DT.strptime(d_INI['SDATE'], '%Y%m%d'), '%Y-%m-%d')
dx = dy = float(d_INI['CELLSIZE'])

d_Pa_B = U.get_MdlN_Pa(MdlN_B)
d_INI_B = U.INI_to_d(d_Pa_B['INI'])
SP_date_1st_B = DT.strftime(DT.strptime(d_INI_B['SDATE'], '%Y%m%d'), '%Y-%m-%d')

In [10]:
SP_date_1st_B, SP_date_last_B = [DT.strftime(DT.strptime(d_INI_B[f'{i}'], '%Y%m%d'), '%Y-%m-%d') for i in ['SDATE', 'EDATE']]

In [22]:
date = DT.strptime("1995-12-31", '%Y-%m-%d')

# 1. Work

In [59]:
DF_1, DF_1_Tot = flopy.utils.Mf6ListBudget(PJ(d_Pa['Sim_In'], 'imported_model.lst')).get_dataframes()

In [60]:
DF_2, DF_2_Tot = flopy.utils.Mf6ListBudget(d_Pa_B['LST_Mdl']).get_dataframes()

In [61]:
start_date = pd.to_datetime(SP_date_1st)
for DF in [DF_1, DF_1_Tot, DF_2, DF_2_Tot]:
    DF.index = pd.date_range(start=start_date, periods=len(DF), freq='D')

In [62]:
S_1 = DF_1.loc[DF_1.index == date]
S_2 = DF_2.loc[DF_2.index == date]

In [63]:
DF = pd.DataFrame(data={MdlN: S_1.squeeze(), MdlN_B: S_2.squeeze()})

In [64]:
sorted_i = [col for col in DF.index if '_IN' in col] + [col for col in DF.index if '_OUT' in col] + [col for col in DF.index if '_OUT' not in col and '_IN' not in col]
DF = DF.reindex(index=sorted_i)

In [65]:
DF['Diff'] = DF[MdlN] - DF[MdlN_B]

In [66]:
DF = DF.replace([np.inf, -np.inf], np.nan).round(0).astype("Int64")

In [71]:
DF['Diff_%'] = DF.apply(lambda x: x['Diff'] / x[MdlN_B] * 100 if pd.notnull(x['Diff']) and x[MdlN_B] != 0 else np.nan, axis=1)

In [72]:
# Replace infinities, convert to nullable Int64, and display missing values as '-'
DF = DF.replace([np.inf, -np.inf], np.nan).round(0).astype("Int64")
DF.style.format(na_rep='-')

,NBr40,NBr43,Diff,Diff_%
CHD_IN,24233,21900,2333,11
DRN-TO-MVR2_IN,0,-,-,-
DRN-TO-MVR3_IN,0,-,-,-
DRN-TO-MVR_IN,0,-,-,-
DRN2_IN,0,-,-,-
DRN3_IN,0,-,-,-
DRN_IN,0,0,0,-
RCH_IN,109023,98398,10625,11
RIV_IN,-,1,-,-
SFR_IN,3023,-,-,-


In [ ]:
DF.to_excel(PJ(d_Pa['PoP_Out_MdlN'], f'WB_{MdlN}_m_{MdlN_B}.xlsx'), index=True, na_rep='-')